In [1]:
import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.optim import Optimizer, Adam
from typing import Optional,Callable,Any
from torchvision.models.resnet import ResNet,_resnet,conv1x1,conv3x3
from torchvision.models import WeightsEnum
import torch
from torch import nn
import torch.nn.functional as F
from torchmetrics.functional.classification.accuracy import accuracy
import pytorch_lightning as pl
import numpy as np
import logging
import sys
sys.path.append("C:\Files\Github Repo\TrainLPS\learn_poly_sampling")

In [2]:
from models import get_available_models, get_model

In [3]:
get_available_models()

['basic',
 'ResNet18Custom',
 'ResNet50Custom',
 'ResNet101Custom',
 'ResNetLite50Type1Custom',
 'ResNetLite50Type2Custom']

In [4]:
from layers import get_pool_method

In [5]:
extras_model = {
    'maxpool_zpad': True,
    'swap_conv_pool': False,
    'inc_conv1_support': True,
    'apply_maxpool': True,
    'ret_prob': True,
    'logits_channels': None,
    'conv1_stride': False,
}

class flag_struct():
  def __init__(self):
    self.dryrun = False
    self.pool_method = 'LPS'
    self.logits_model = 'LPSLogitLayers'
    self.swap_conv_pool = False
    self.maxpool_zpad = False
    self.LPS_pad = 'circular'
    self.LPS_debug = False
    self.LPS_convex = False
    self.pool_k = 2
    self.antialias_mode = 'LowPassFilter'
    self.antialias_size = 3
    self.antialias_padding = 'same'
    self.antialias_padding_mode = "circular"
    self.antialias_group = 8
    self.selection_noantialias = False
    self.LPS_gumbel = False
    self.LPS_train_convex = False


FLAGS = flag_struct()
pool_layer = get_pool_method(FLAGS.pool_method, FLAGS)
model = get_model('ResNetLite50Type2Custom')(224, 1000, extras_model = extras_model, pooling_layer=pool_layer)

In [6]:
model(torch.rand(2,3,224,224))

tensor([[-6.4657, -7.1485, -7.2900,  ..., -7.6158, -7.7331, -7.0894],
        [-6.3186, -7.2687, -7.5850,  ..., -8.3226, -7.8279, -6.9511]],
       grad_fn=<LogSoftmaxBackward0>)

In [7]:
model

ResNetLite50Type2Custom(
  (core): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): Sequential(
      (0): PolyphaseInvariantDown2D(
        (component_selection): LPS(
          (get_logits): LPSLogitLayersV2(
            (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same, padding_mode=circular)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same, padding_mode=circular)
            (relu): ReLU()
          )
        )
      )
      (1): ZeroPad2d((0, 1, 0, 1))
      (2): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
      (3): PolyphaseInvariantDown2D(
        (component_selection): LPS(
          (get_logits): LPSLogitLayersV2(
            (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same, padding_m